In [34]:
import re

In [39]:
data = open('data/day14.txt', 'r').read()

In [6]:
example = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""

In [26]:
def move_rock_north(grid, row, col):
    if row == 0:
        return (0, col)
    
    if grid[row - 1][col] not in ['#', 'O']:
        return move_rock_north(grid, row - 1, col)
    else:
        return (row, col)

In [42]:
def move_all_rocks_north(grid):
    new_grid = [['#' if grid[row][col] == '#' else '.' for col in range(len(grid[row]))] for row in range(len(grid))]

    for row in range(len(grid)):
        for col in range(len(grid[row])):
            if grid[row][col] == 'O':
                (moved_row, moved_col) = move_rock_north(new_grid, row, col)
                new_grid[moved_row][moved_col] = 'O'
    
    return new_grid

In [240]:
def part1(data):
    grid = [list(line) for line in data.split('\n')]
    new_grid = move_all_rocks_north(grid)
    
    total_load = 0
    max_load = len(new_grid)
    for row_index, row in enumerate(new_grid):
        load = max_load - row_index
        num_rocks = len(re.findall('O', ''.join(row)))
        total_load += load * num_rocks
    
    return total_load
part1(data)        

111339

In [108]:
def flip_layout(layout):
    """
    Transpose our layout storing the rows and columns of coordinates
    """
    flipped_layout = {}
    for row in layout:
        for col in layout[row]:
            if col in flipped_layout:
                flipped_layout[col].append(row)
            else: 
                flipped_layout[col] = [row]
                
    return flipped_layout

In [262]:
def shift_stones(rocks, stones, direction, col_bound):
    """
    Shift stones as close as possible to their right bounds, whether that's the true bound or a rock
    """
    new_stones = {}
    for row_index, stone_row in stones.items():
        new_stone_row = []
        remaining_stones = len(stone_row)
        # Add additional bounds for either end of the row
        row_rocks = [-1] + rocks.get(row_index, []) + [col_bound]
        # Shift stones towards their left bounds
        for bound_index in range(1, len(row_rocks)):
            left_bound, right_bound = row_rocks[bound_index - 1], row_rocks[bound_index]
            # Check for shifting stones when there's a gap between bounds
            if right_bound - left_bound > 1:
                moving_stones = [col for col in stone_row if left_bound < col < right_bound]
                if len(moving_stones) > 0:
                    if direction == 'left':
                        moved_stones = [(left_bound + 1) + stone for stone in range(len(moving_stones))]
                    else:
                        moved_stones = [(right_bound - 1) - stone for stone in range(len(moving_stones))]  # -1 is so we move to before the right bound                     
                    new_stone_row += moved_stones
                    remaining_stones -= len(moved_stones) 
                    if remaining_stones == 0:  # Break when we've shifted all available stones in the row
                        break
        new_stones[row_index] = new_stone_row
    
    return new_stones

In [274]:
def print_grid(rocks, stones, max_row, max_col):
    """
    Construct the grid from the rocks and stones at a particular step and print it
    """
    grid = [['.' for col in range(max_col)] for row in range(max_row)]
    for row, col_list in rocks.items():
        for col in col_list:
            grid[row][col] = '#'
    for row, col_list in stones.items():
        for col in col_list:
            grid[row][col] = 'O'
    
    for row in grid:
        print(''.join(row))

    return grid

In [337]:
def spin_cycle(rocks, stones, row_bound, col_bound):
    """
    Perform a full spin cycle
    """
    flipped_rocks = flip_layout(rocks)
    shifted_stones = stones

    # Shift north
    shifted_stones = shift_stones(flipped_rocks, flip_layout(shifted_stones), 'left', row_bound)
    # Shift west
    shifted_stones = shift_stones(rocks, flip_layout(shifted_stones), 'left', col_bound)    
    # Shift south
    shifted_stones = shift_stones(flipped_rocks, flip_layout(shifted_stones), 'right', row_bound)
    # Shift east
    shifted_stones = shift_stones(rocks, flip_layout(shifted_stones), 'right', col_bound)

    return shifted_stones

In [338]:
def calculate_load(stones, row_bound):
    """
    Calculate total load
    """
    total_load = 0
    for row_index, row in stones.items():
        load = row_bound - row_index
        num_rocks = len(row)
        total_load += load * num_rocks
    
    return total_load

In [504]:
def find_load_loop(rocks, stones, row_bound, col_bound, num_cycles):
    """
    Find the point at which there is a loop in the load pattern
    """
    visited = []
    prior_visits = {}
    loops = {}
    shifted_stones = stones
    for cycle in range(num_cycles):
        shifted_stones = spin_cycle(rocks, shifted_stones, row_bound, col_bound)
        load = calculate_load(shifted_stones, row_bound)
        
        for key in loops:
            loops[key].append(load)
        
        if load not in loops:
            loops[load] = []
        else:
            if load not in prior_visits:
                prior_visits[load] = [sorted(loops[load])]
            else:
                if sorted(loops[load]) in prior_visits[load] and len(set(loops[load])) > 1:
                    unique_loop_loads = [loop_load for loop_load in loops[load] if loops[load].count(loop_load) == 1]
                    if all([sorted(loops[load]) in prior_visits[unique_loop_load] for unique_loop_load in unique_loop_loads]):
                        pattern_index = loops[load].index(load)
                        pattern = loops[load][pattern_index:] + loops[load][:pattern_index]
                        return (cycle + 1, pattern)
                prior_visits[load].append(sorted(loops[load]))
                loops[load] = []             
        
        

In [509]:
def part2(data, num_cycles):
    grid = [list(line) for line in data.split('\n')]
    row_bound, col_bound = len(grid), len(grid[0])
    rocks = {row: [col for col in range(len(grid[row])) if grid[row][col] == '#'] for row in range(len(grid)) if len(re.findall('#', ''.join(grid[row]))) != 0}
    stones = {row: [col for col in range(len(grid[row])) if grid[row][col] == 'O'] for row in range(len(grid)) if len(re.findall('O', ''.join(grid[row]))) != 0}
    loop_cycle, loop = find_load_loop(rocks, stones, row_bound, col_bound, num_cycles)
    
    return loop[(num_cycles - loop_cycle) % len(loop)]

part2(data, 1000000000)

93736